In [6]:
import re, sys, copy, numpy as np
from math import sqrt
from aoclib import list2string
from collections import defaultdict
from itertools import combinations
from operator import add, mul

lines = open('19.txt').read().strip()

program = [int(x) for x in lines.split(",")]

In [7]:
def icc(program, inputs = None): # intcode computer
    memory = program.copy()
    for j in range(10000 - len(memory)):
        memory.append(0)
                      
    output = []
    alt_par_modes = False # parameter modes other than position mode are enabled / disabled by this boolean
    relative_base = 0

    i = 0
    while True:
        if alt_par_modes:
            alt_par_modes = False
            instruction_header = str(memory[i]).rjust(5, "0")
            opcode = int(instruction_header[-2:])
            par_modes = [int(x) for x in instruction_header[-3::-1]]
        else:
            opcode = memory[i]
            par_modes = [0, 0, 0]

        if opcode == 99:
            break
            
        idx1 = i + 1 if par_modes[0] == 1 else memory[i + 1]
        idx2 = i + 2 if par_modes[1] == 1 else memory[i + 2]
        idx3 = i + 3 if par_modes[2] == 1 else memory[i + 3]
        if par_modes[0] == 2:
            idx1 += relative_base
        if par_modes[1] == 2:
            idx2 += relative_base
        if par_modes[2] == 2:
            idx3 += relative_base
        
        match opcode:
            case 1 | 2:
                memory[idx3] = memory[idx1] + memory[idx2] if opcode == 1 else memory[idx1] * memory[idx2]
                i += 4
            case 3:
                if inputs:
                    input_ = int(inputs.pop(0))
                else:
                    input_ = int(input('Please provide some input: '))
                memory[idx1] = input_
                i += 2
            case 4:
                output.append(memory[idx1])
                i += 2
            case 5:
                i = memory[idx2] if memory[idx1] != 0 else i + 3
            case 6:
                i = memory[idx2] if memory[idx1] == 0 else i + 3
            case 7:
                memory[idx3] = 1 if memory[idx1] < memory[idx2] else 0
                i += 4
            case 8:
                memory[idx3] = 1 if memory[idx1] == memory[idx2] else 0
                i += 4
            case 9:
                relative_base += memory[idx1]
                i += 2
            case _:
                alt_par_modes = True
                continue
    return output


In [5]:
# Part 1
grid = ""
for i in range(50):
    for j in range(50):
        output = icc(program, [i, j])
        grid += '#' if output[0] == 1 else '.'
    grid += "\n"
print(grid.count('#'))
print(grid)

NameError: name 'program' is not defined

In [21]:
# Part 2
sz = 2000
grid = [['.' for i in range(sz)] for j in range(sz)]
finished = False
last_j = 0
for i in range(sz):
    if i % 100== 0:
        print(i)
    j = last_j
    while True:
        output = icc(program, [i, j])
        if output[0] == 1:
            grid[i][j] = '#'
            if grid[i][j - 1] == '.':
                last_j = j
                if i > 100: 
                    if grid[i - 99][j + 99] == '#':
                        print("found necessary beam size")
                        finished = True
                        break
        elif grid[i][j - 1] == '#':
            break
        elif j >= sz - 1:
            break
        j += 1
    if finished:
        break
                
print((i-99)*10000 + j)

0
100
200
300
400
500
600
700
800
900
1000
1100
1
i j .
i j-1 .
i - 99 j + 99 #
found necessary beam size
104796
10010825


In [22]:
import numpy as np
from sklearn.linear_model import LinearRegression 

input = open('19.txt', 'r').read().split(',')
input = np.array([int(i) for i in input])
c = {}
def is_b(x, y):
  if (x, y) in c:
    return c[x, y]
  c[x, y] = icc(program, inputs=[x, y])[0]
  return c[x, y]

s = 0
X = []
Y = []
for y in range(50):
  for x in range(50):
    o = is_b(x, y)
    s += o
    if o:
      X.append(x)
      Y.append(y)
    print('#' if o else '.', end=' ')
  print('')
print(s)

X = np.array(X).reshape(-1, 1)
Y = np.array(Y)
model = LinearRegression().fit(X, Y)

def f(low, x, high=9999):
  while(high - low > 1):
    y = (low + high) // 2
    if is_b(x, y):
      low = y
    else:
      high = y
  return low

def is_v(x):
  y = int(model.predict(np.array([x]).reshape(-1, 1)))
  y_high = f(y, x)
  if is_b(x, y_high - M + 1):
    y_c = y_high - M + 1
    if is_b(x + M - 1, y_c):
      return True, y_c
  return False, -1

low = 0
high = 9999
M = 100

while(high - low > 1):
  x = (low + high) // 2
  if is_v(x)[0]:
    high = x
  else:
    low = x

x = high
b_y = 0  
for i in range(10, 0, -1):
  v, y = is_v(high - i)
  if v:
    x = high - i
    b_y = y

print(f'x: {x} and y: {b_y}, ans: {10000*x + b_y}')

# . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . # # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . # # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . # # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . # # # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

/var/folders/m7/fb2rmjkn5xl1dxf23c4zfy0h0000gp/T/ipykernel_45263/877768459.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y = int(model.predict(np.array([x]).reshape(-1, 1)))
/var/folders/m7/fb2rmjkn5xl1dxf23c4zfy0h0000gp/T/ipykernel_45263/877768459.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y = int(model.predict(np.array([x]).reshape(-1, 1)))
/var/folders/m7/fb2rmjkn5xl1dxf23c4zfy0h0000gp/T/ipykernel_45263/877768459.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Depre